<a href="https://colab.research.google.com/github/maxjing97/plm_model_studies2025/blob/main/model_running.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running ESM fold to test

We can run esmfold from hugging face Here

In [ ]:
import esm
import torch

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device =  torch.device("cpu")
# Print the device being used
print(f'Using device: {device}')

Using device: cpu


In [ ]:
# Load model directly
from transformers import AutoTokenizer, EsmForProteinFolding
from accelerate.test_utils.testing import get_backend

tokenizer = AutoTokenizer.from_pretrained("facebook/esmfold_v1")
model = EsmForProteinFolding.from_pretrained("facebook/esmfold_v1", low_cpu_mem_usage=True)

print("current device", device)
model = model.to(device)
model.eval()



/home/maxjing/miniconda3/envs/esmfold/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of EsmForProteinFolding were not initialized from the model checkpoint at facebook/esmfold_v1 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


current device cpu


EsmForProteinFolding(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 2560, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-35): 36 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=2560, out_features=2560, bias=True)
              (key): Linear(in_features=2560, out_features=2560, bias=True)
              (value): Linear(in_features=2560, out_features=2560, bias=True)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
          )
          (intermediate): EsmIntermediate(
            (dense): Linear(in_feat

Encoding part:

In [ ]:

# Example protein sequence
test_protein = "MGAGASAEEKHSRELEKKLKEDAEKDARTVKLLLLGAGESGKSTIVKQMKIIHQDGYSLEECLEFIAIIYGNTLQSILAIVRAMTTLNIQYGDSARQDDARKLMHMADTIEEGTMPKEMSDIIQRLWKDSGIQACFERASEYQLNDSAGYYLSDLERLVTPGYVPTEQDVLRSRVKTTGIIETQFSFKDLNFRMFDVGGQRSERKKWIHCFEGVTCIIFIAALSAYDMVLVEDDEVNRMHESLHLFNSICNHRYFATTSIVLFLNKKDVFFEKIKKAHLSICFPDYDGPNTYEDAGNYIKVQFLELNMRRDVKEIYSHMTCATDTQNVKFVFDAVTDIIIKENLKDCGLF"

inputs = tokenizer([test_protein], return_tensors="pt", add_special_tokens=False)['input_ids']
input = input.to(device)

In [ ]:
print(inputs)

tensor([[12,  7,  0,  7,  0, 15,  0,  6,  6, 11,  8, 15,  1,  6, 10,  6, 11, 11,
         10, 11,  6,  3,  0,  6, 11,  3,  0,  1, 16, 19, 11, 10, 10, 10, 10,  7,
          0,  7,  6, 15,  7, 11, 15, 16,  9, 19, 11,  5, 12, 11,  9,  9,  8,  5,
          3,  7, 18, 15, 10,  6,  6,  4, 10,  6, 13,  9,  0,  9,  9, 18,  7,  2,
         16, 10,  5, 15,  9, 10,  0,  9, 19,  1,  0, 12, 16, 16, 10,  2,  9,  5,
         18,  7,  3, 15,  0,  1,  5,  3,  3,  0,  1, 11, 10, 12,  8, 12,  0,  3,
         16,  9,  6,  6,  7, 16, 12, 14, 11,  6, 12, 15,  3,  9,  9,  5,  1, 10,
         17, 11,  3, 15,  7,  9,  5,  0,  4, 13,  6,  1,  0, 15,  6, 18,  5, 10,
          2,  3, 15,  0,  7, 18, 18, 10, 15,  3, 10,  6,  1, 10, 19, 16, 14,  7,
         18, 19, 14, 16,  6,  5,  3, 19, 10,  1, 15,  1, 19, 11, 16, 16,  7,  9,
          9,  6, 16,  5, 13, 15, 13, 11,  3, 10,  2, 13,  1, 12, 13,  3, 19,  7,
          7,  5,  1, 15,  6,  1, 11, 11, 17,  9,  8,  4, 13,  6,  7, 19, 16,  4,
          9,  9, 13,  9,  0,

In [ ]:
# Run inference
with torch.no_grad():
    outputs = model(inputs)

# Get the predicted 3D coordinates
coords = outputs.coordinates[0].cpu().numpy()

print("Predicted coordinates shape:", coords.shape)

KeyboardInterrupt: 